In [3]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [4]:
def swish(x,beta=1.):
    return x*tf.nn.sigmoid(beta*x)

class DNN(Model):
    def __init__(self,hidden_dim,output_dim):
        super().__init__()
        self.l1=Dense(hidden_dim,activation=lambda x:swish(x,beta=10.))
        self.l2=Dense(hidden_dim,activation=lambda x:swish(x,beta=10.))
        self.l3=Dense(hidden_dim,activation=lambda x:swish(x,beta=10.))
        self.l4=Dense(output_dim,activation='softmax')
        self.ls=[self.l1,self.l2,self.l3,self.l4]
        
    def call(self,x):
        for layer in self.ls:
            x=layer(x)
        return x

In [5]:
mnist=datasets.mnist
(x_train,t_train),(x_test,t_test)=mnist.load_data()

x_train=(x_train.reshape(-1,784)/255).astype(np.float32)
x_test=(x_test.reshape(-1,784)/255).astype(np.float32)
t_train=np.eye(10)[t_train].astype(np.float32)
t_test=np.eye(10)[t_test].astype(np.float32)

In [6]:
model=DNN(200,10)

In [7]:
criterion=losses.CategoricalCrossentropy()
optimizer=optimizers.SGD(learning_rate=0.1)
train_loss=metrics.Mean()
train_acc=metrics.CategoricalAccuracy()

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds=model(x)
        loss=compute_loss(preds,t)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss)
    train_acc(t,preds)
    return loss

epochs=30
batch_size=100
n_batches=x_train.shape[0]//batch_size

for epoch in range(epochs):
    x_,t_=shuffle(x_train,t_train)
    
    for batch in range(n_batches):
        start=batch*batch_size
        end=start+batch_size
        train_step(x_[start:end],t_[start:end])
    
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch,train_loss.result(),train_acc.result()))


epoch: 0, loss: 3.827, acc: 0.767
epoch: 1, loss: 3.111, acc: 0.810
epoch: 2, loss: 2.845, acc: 0.826
epoch: 3, loss: 2.698, acc: 0.834
epoch: 4, loss: 2.611, acc: 0.839
epoch: 5, loss: 2.557, acc: 0.842
epoch: 6, loss: 2.511, acc: 0.845
epoch: 7, loss: 2.489, acc: 0.846
epoch: 8, loss: 2.471, acc: 0.847
epoch: 9, loss: 2.491, acc: 0.846
epoch: 10, loss: 2.513, acc: 0.845
epoch: 11, loss: 2.552, acc: 0.842
epoch: 12, loss: 2.677, acc: 0.834
epoch: 13, loss: 2.779, acc: 0.828
epoch: 14, loss: 3.009, acc: 0.814
epoch: 15, loss: 3.413, acc: 0.789
epoch: 16, loss: 3.696, acc: 0.771
epoch: 17, loss: 4.046, acc: 0.749
epoch: 18, loss: 4.498, acc: 0.721
epoch: 19, loss: 4.929, acc: 0.695
epoch: 20, loss: 5.384, acc: 0.666
epoch: 21, loss: 5.797, acc: 0.641
epoch: 22, loss: 6.174, acc: 0.617
epoch: 23, loss: 6.519, acc: 0.596
epoch: 24, loss: 6.838, acc: 0.576
epoch: 25, loss: 7.131, acc: 0.558
epoch: 26, loss: 7.403, acc: 0.541
epoch: 27, loss: 7.655, acc: 0.525
epoch: 28, loss: 7.890, acc: 0

In [8]:
test_loss=metrics.Mean()
test_acc=metrics.CategoricalAccuracy()

def test_step(x,t):
    preds=model(x)
    loss=compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return loss

test_step(x_test,t_test)

print('test_loss: {:.3f}, test_acc:{:.3f}'.format(test_loss.result(),test_acc.result()))

test_loss: 14.490, test_acc:0.101
